## Import

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import re
from database import Database
import pandas as pd
import json
import time

from database import Database
import functions

In [2]:
#init
database = Database()

#create tables
database.create_table_championships()
database.create_table_teams()
database.create_table_players()
database.create_table_participations()
database.create_table_contracts()
database.create_table_matches()
database.create_table_matches_teams()
database.create_table_goals()

## Scrapping Alexandre

In [9]:
def connect(url):
    url = url
    response = requests.get(url)
    if response.ok:
      soup = BeautifulSoup(response.text,'html.parser')
    else:
      print(f'L\'erreur {response.status_code} s\'est produite')
    return soup

def replace_com_ls(ls, old_value, new_value):
    return [str(k).replace(old_value,new_value) for k in ls]

In [10]:
home = connect('https://www.lequipe.fr/Football/ligue-1/page-classement-equipes/general')

In [11]:
ls_teams = []
for link in home.find_all('a', attrs={'class':'table__link'}):
    if link.has_attr('href'):
        ls_teams.append('https://www.lequipe.fr' + link['href'])

In [12]:
name,country,start_year,end_year = "Ligue 1 UberEats", "France", 2020, 2021
championship_id = database.insert_table_championships(name,country,start_year,end_year)

In [13]:
for link in ls_teams:
    link_teams = connect(str(link))
    name = link_teams.td.strong.text
    city = link_teams.section.h1.text
    if "(" in city:
        city = city.split()
        del city[1]
        city= city[0]
    else:
        city
    date_creation = link_teams.select('td > strong')[1].text
    president = link_teams.select('td > strong')[2].text
    coach_name = link_teams.select('td > a > strong')[0].text
    
    team_id = database.insert_table_teams(name,city,coach_name,president,date_creation)
    database.insert_table_participations(team_id,championship_id)

In [14]:
for link in ls_teams:
    link_teams = connect(str(link))
    team_name = link_teams.select("h1.nom_sportif")
    team_name = [i.text for i in team_name]
    if "(" in team_name[0]:
        team_name = team_name[0].split()
        del team_name[1]
        team_name
    else:
        team_name[0]

    name=link_teams.select("td.nom>a")
    name = [i.text for i in name]
    del name[-1]
    name = replace_com_ls(name, "      ", '')
    name = replace_com_ls(name, " ", '')
    nationality = link_teams.select("td.pays")
    nationality = [i.text for i in nationality]    
    birthdate = link_teams.find_all("td", attrs={'class':'ddn'})
    birthdate = [i.text for i in birthdate]
    position =  link_teams.find_all("td", attrs={'class':'poste'})
    position =  [i.text for i in position]

    for name,birthdate,position,nationality in zip(name,birthdate,position,nationality):
        team_id = database.get_team_id(team_name[0])
        player_id = database.insert_table_players(name,position,birthdate,nationality)
        database.insert_table_contracts(player_id,team_id)

## Scrapping Flo

In [15]:
page = requests.get('https://iphdata.lequipe.fr/iPhoneDatas/EFR/STD/ALL/V1/Football/CalendarList/CompetitionPhase/ligue-1/current/1re-journee.json')

In [16]:
pages = [page.json()]
for i in range(2, 39, 1):
    request = requests.get(f"https://iphdata.lequipe.fr/iPhoneDatas/EFR/STD/ALL/V1/Football/CalendarList/CompetitionPhase/ligue-1/current/{i}e-journee.json")
    pages.append(request.json())

In [61]:
df_scrap = pd.DataFrame(columns=['City', 'Date', 'Hour', 'Winner', 'Home','HomeGoals', 'Visitor', 'VisitorGoals', 'Week', 'Web'])

In [62]:
count = 0
for k in range(len(pages)):
    count+=1
    jpage = pages[k]
    for i in range(len(jpage['items']) - 1):
        for j in range (len(jpage['items'][i]['items'])):
            event = jpage['items'][i]['items'][j]['event']
            city_match = event['lieu']['ville']
            date_match = event['date']
            winner = event['specifics']['vainqueur']
            visitor_team = event['specifics']['exterieur']['equipe']['nom']
            home_team = event['specifics']['domicile']['equipe']['nom']
            visitor_goals = event['specifics']['score']['exterieur']
            home_goals = event['specifics']['score']['domicile']
            link_web = event['lien_web']
            df_scrap = df_scrap.append({'City' : city_match, 'Date' : date_match, 'Winner' : winner, 'Visitor' : visitor_team, 'Home' : home_team, 'VisitorGoals' : visitor_goals, 'HomeGoals' : home_goals, 'Week' : count, 'Web' : link_web}, ignore_index=True)

In [63]:
df_scrap['Date'] = [i.split('T') for i in df_scrap['Date']]
df_scrap['Hour'] = [i[1] for i in df_scrap['Date']]
df_scrap['Hour'] = [i.split('+') for i in df_scrap['Hour']]
df_scrap['Hour'] = [i[0] for i in df_scrap['Hour']]
df_scrap['Date'] = [i[0] for i in df_scrap['Date']]
df_scrap['Date'] = df_scrap['Date'].str.replace("-", " ")
df_scrap['Date'] = pd.to_datetime(df_scrap['Date'])
df_scrap['Date'] = df_scrap['Date'].dt.strftime('%Y %m %d')

In [64]:
df_scrap

,City,Date,Hour,Winner,Home,HomeGoals,Visitor,VisitorGoals,Week,Web
0,Bordeaux,2020 08 21,19:00:00,ex-aequo,Bordeaux,0,Nantes,0,1,https://www.lequipe.fr/Football/match-direct/l...
1,Dijon,2020 08 22,17:00:00,exterieur,Dijon,0,Angers,1,1,https://www.lequipe.fr/Football/match-direct/l...
2,Lille,2020 08 22,21:00:00,ex-aequo,Lille,1,Rennes,1,1,https://www.lequipe.fr/Football/match-direct/l...
3,Monaco,2020 08 23,13:00:00,ex-aequo,Monaco,2,Reims,2,1,https://www.lequipe.fr/Football/match-direct/l...
4,Lorient,2020 08 23,15:00:00,domicile,Lorient,3,Strasbourg,1,1,https://www.lequipe.fr/Football/match-direct/l...
...,...,...,...,...,...,...,...,...,...,...
375,Rennes,2021 05 23,21:00:00,domicile,Rennes,2,Nîmes,0,38,https://www.lequipe.fr/Football/match-direct/l...
376,Nantes,2021 05 23,21:00:00,exterieur,Nantes,1,Montpellier,2,38,https://www.lequipe.fr/Football/match-direct/l...
377,Saint-Étienne,2021 05 23,21:00:00,exterieur,Saint-Étienne,0,Dijon,1,38,https://www.lequipe.fr/Football/match-direct/l...
378,Reims,2021 05 23,21:00:00,exterieur,Reims,1,Bordeaux,2,38,https://www.lequipe.fr/Football/match-direct/l...


In [65]:
df_match = df_scrap[['City', 'Date', 'Hour']]
df_home_team = df_scrap[['Home', 'HomeGoals']]
df_visitor_team = df_scrap[['Visitor', 'VisitorGoals']]

In [66]:
for i in range(len(df_match)):
    time.sleep(0.5)
    place = df_match.loc[i]['City']
    date = df_match.loc[i]['Date']
    hour = df_match.loc[i]['Hour']
    date = date.replace("-"," ")
    #print(date,place,hour)
    link_weather = functions.create_link_weather(place, date)
    #print(link_weather)
    tmax, rainfall = functions.scrapmeteo(link_weather)
    match_id = database.insert_table_matches(championship_id,date,place,rainfall,tmax)

    #matches_teams Dom
    name_city = df_home_team.loc[i]['Home']
    team_goal = df_home_team.loc[i]['HomeGoals']
    home=True
    team_id = database.get_team_id(name_city)
    #print(match_id, team_id, home, team_goal)
    database.insert_table_matches_teams(match_id, team_id, home, team_goal)
        
    #matches_teams Ext
    name_city = df_visitor_team.loc[i]['Visitor']
    team_goal = df_visitor_team.loc[i]['VisitorGoals']
    home=False
    team_id = database.get_team_id(name_city)
    #print(match_id, team_id, home, team_goal)
    database.insert_table_matches_teams(match_id, team_id, home, team_goal)
        
    #goals
    page = requests.get(df_scrap.loc[i]['Web'])
    soup = BeautifulSoup(page.text, 'html.parser')
    goalers = soup.find_all("span", {"class": "Scoreboard__scorer"})
    for j in range(len(goalers)):
        goal_type = soup.find_all("div", {"class": "Scoreboard__goals"})
        goal_type = goal_type[j].find_all('span')
        if ((goal_type[0].text or goal_type[2].text) != '(csc)' and (goal_type[0].text or goal_type[2].text) != '(s.p.)'):
            #print(f'Journée : {df_scrap.loc[i].Week}, Buteur : {goalers[j].text}, type : but')
            player_name = goalers[j].text.replace(" ","")
            #print(player_name)
            player_id = database.get_player_id(player_name)
            goal_type = "normal"
            database.insert_table_goals(player_id, match_id, goal_type)                
        elif ((goal_type[0].text or goal_type[2].text) == '(s.p.)'):
            #print(f'Journée : {df_scrap.loc[i].Week}, Buteur : {goalers[j].text}, type : penalty')
            player_name = goalers[j].text.replace(" ","")
            #print(player_name)
            player_id = database.get_player_id(player_name)
            goal_type = "penalty"
            database.insert_table_goals(player_id, match_id, goal_type)
        else:
            #print(f'Journée : {df_scrap.loc[i].Week}, Buteur : {goalers[j].text}, type : contre son camp')
            player_name = goalers[j].text.replace(" ","")
            #print(player_name)
            player_id = database.get_player_id(player_name)
            goal_type = "contre son camp"
            database.insert_table_goals(player_id, match_id, goal_type)